<a href="https://colab.research.google.com/github/danfolador/recpadroes/blob/main/RecPad_Exercicio09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#==============================================================================================
# INICIALIZACAO
#==============================================================================================
import pandas as pd
import numpy as np
import time

#Estimadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV

#Preprocessamento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#Voting
from sklearn.ensemble import VotingClassifier

#Stacking
from sklearn.ensemble import StackingClassifier

In [4]:
#==============================================================================================
# BANCO DE DADOS
#==============================================================================================
#FONTE: https://archive.ics.uci.edu/ml/machine-learning-databases/00229/

url='https://archive.ics.uci.edu/ml/machine-learning-databases/00229/Skin_NonSkin.txt'
bd = pd.read_csv(url, sep= '\t') 

X  = bd.iloc[:, 0:-1]
y = bd.iloc[:, -1]

X = StandardScaler().fit_transform(X)
y = pd.DataFrame.to_numpy(y) 

X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=1)

print('=========================')
print('BANCO DE DADOS')
print('=========================')
print('shape(X)=',np.shape(X))
print('shape(y)=',np.shape(y))
print('n classes=',len(set(y)))

BANCO DE DADOS
shape(X)= (245056, 3)
shape(y)= (245056,)
n classes= 2


In [6]:
#==============================================================================================
# VOTING
#==============================================================================================

#Use the key for the classifier followed by '__' and the attribute
params_voting = {'voting': ['hard','soft'],
                 'knn__n_neighbors': [4,5,6],
                 'extratrees__criterion': ['gini','entropy'],
                 'randomforest__max_depth': [7,8,9]
                  }


voting = VotingClassifier([('knn',KNeighborsClassifier()),
                           ('extratrees',ExtraTreesClassifier(random_state=1)),
                           ('randomforest',RandomForestClassifier(random_state=1))
                           ])

grid_voting = GridSearchCV(estimator=voting,
                    param_grid=params_voting,
                    n_jobs=-1)

start=time.time()
modelo=grid_voting
modelo.fit(X_tr,y_tr)
ypred = modelo.predict(X_te)
hits = ypred == y_te           
score = sum(hits)/len(hits)

stop=time.time()
t=stop-start

print('score voting= ',score)
print('t(s)= ',t)
print('t(min)= ',t/60)
print('-------------------')

score voting=  0.9994613476103421
t(s)=  1731.7221610546112
t(min)=  28.86203601757685
-------------------


In [7]:
#==============================================================================================
# STACKING
#==============================================================================================
#Usa StackingClassifier com GridSearch(VotingClassifier)

params_stacking = {'cv': [2,3],
                 'knn__n_neighbors': [4,5,6],
                 'extratrees__criterion': ['gini','entropy'],
                 'randomforest__max_depth': [7,8,9]
                  }


stacking = StackingClassifier([('knn',KNeighborsClassifier()),
                               ('extratrees',ExtraTreesClassifier(random_state=1)),
                               ('randomforest',RandomForestClassifier(random_state=1))
                               ])


grid_stacking = GridSearchCV(estimator=stacking,
                      param_grid=params_stacking,
                      n_jobs=-1)

start=time.time()
modelo=grid_stacking
modelo.fit(X_tr,y_tr)
ypred = modelo.predict(X_te)
hits = ypred == y_te           
score = sum(hits)/len(hits)

stop=time.time()
t=stop-start

print('score stacking= ',score)
print('t(s)= ',t)
print('t(min)= ',t/60)
print('-------------------')

score stacking=  0.9995756072081483
t(s)=  4276.747447729111
t(min)=  71.27912412881851
-------------------
